![LogoTremplin](images/tremplinColorSmall.png)

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.

![Licence](images/Licence.jpg)

Auteurs : G. Vidal, C-H. Eyraud, E. le Jan

------------------------------------------------------------

# La detection de présence : Projet EyeKAM

Projet conduit par trois élèves au Lycée Camille Saint Saëns, encadrés par Marie Pierre Cribier, Pascal Delacour, Eric Le Jan. 

[la page du capteur](https://learn.adafruit.com/pir-passive-infrared-proximity-motion-sensor?view=all)

le capteur recto et verso

|||
|:--:|:--:|
|![recto](images/capteurPIRmotionFace.png)|![verso](images/capteurPIRmotion.png)| 

[la page utilisée](https://learn.adafruit.com/adafruits-raspberry-pi-lesson-12-sensing-movement) pour mettre en place ce petit programme

[La page utilisée](https://openclassrooms.com/courses/apprenez-a-programmer-en-python/le-temps) pour le module time

[La page utilisée](https://docs.python.org/3/library/shutil.html#shutil.copy) pour copier le fichier de capture dans le dossier accessble sur le web.

[La page utilisée](https://www.developpez.net/forums/d1341218/autres-langages/python-zope/general-python/interrompre-boucle-while-l-appuie-d-touche/) pour interrompre proprement le kernel.

Pour les leds l'ipynb utilisé est dans la base CahierdeProgrammes "une led"

Les élèves disposent d'une Raspbery Pi3 d'une bredboard du capteur de mouvement, un capteur de température et un capteur de lumière. 

## Une première approche : Détecter la présence en allumant des leds et afficher une information

L'objectif poursuivi est faire de comprendre qu'il faut une interraction entre la Raspi et l'environnement. 
Le projet doit contenir du code python, on ne peut donc pas se contenter de récupérer le flux streamé de la raspi.

Le premier temps du programme permet de tester l'allumage et l'extinction des leds rouge et verte.

Le programme tourne en boucle l'arrêt se produit à l'interruption du kernel. Le capteur est alimenté en 5 V.

In [ ]:
#On importe la bibliothèque de base de temps, la bibliothèque mraa (compatibilité avec RASPI, Arduino, Intel)
#La bibliothèque RPi.GPIO
import time
import mraa
import RPi.GPIO as io
# décide d'utiliser les Numé&ro des pin inscrits sur le GPIO   
io.setmode(io.BOARD)
# on branche le détecteur sur le pin 18
pir_pin = 18
# On defini le pin 11 pour la led (Rouge)
led = mraa.Gpio(11)
time.sleep(0.5)
#On définit le pin 32 pour la ledverte
ledVerte = mraa.Gpio(32)
#on place le pin 18 en mode IN    
io.setup(pir_pin, io.IN)         
#on place le pin 11 et le pin 32 en mode OUT
status = led.dir(mraa.DIR_OUT)
time.sleep(0.5)
statusVert = ledVerte.dir(mraa.DIR_OUT)
# Onteste les led   
for i in range(5):
    status = led.write(1)
    time.sleep(0.5)
    status = led.write(0)
    time.sleep(0.5)
    statusVert = ledVerte.write(1)
    time.sleep(0.5)
    statusVert = ledVerte.write(0)

# on attend un signal détecté et on affiche une info et une couleur de led à la façon des entrées/sorties de parking à une seule voie. 
valeur = 0
print(io.input(pir_pin))
i=0
while True:
    if io.input(pir_pin):
        status = led.write(1)
        statusVert = ledVerte.write(0)
        print("Il y a quelqu'un !…… ", io.input(pir_pin) ,"…",end='')
    else:
        status = led.write(0)
        statusVert = ledVerte.write(1)
        print("Il n'y a personne !…… ", io.input(pir_pin) ,"…",end='')
    time.sleep(0.5)
    

## Une solution pour arrêter le programme 

Le bouton d'interruption du kernel stoppe le compte. 

In [ ]:
import time
def arrêt():
    x = 0
    while 1:
        try:
            x += 1
        except KeyboardInterrupt:
            print ('Arrêté à :', time.strftime("%A %d %B %Y %H:%M:%S"), "avec x = ",x)
            break
 
arrêt()

## Capture d'une photo avec la picam

Il s'agit ici de capturer une image jpeg, de la déplacer dans le dossier prévu sur la RASPI qui permet de la visualiser en pointant dessus depuis le navigateur. 

Quelques étapes préalables : 

- créer la hiérarchie de dossier sites/media/images/ dans le dossier /var/www/html/

- Donner tous les droits à tout le monde sur le dossier media (dans un premier temps) 

- régler le timelaps pour les captures (exprimé en secondes) 

- régler le nombre de captures souhaitées (entier compris entre 1 et 20) (pas de vérification pour l'instant)


In [2]:
from time import *
from picamera import PiCamera
import shutil
import os
FrequenceCapture = int(input("saisissez la fréquence des captures en secondes entre 1 et 20 secondes : " ))
while FrequenceCapture >20 or FrequenceCapture <1 :
    FrequenceCapture = int(input("saisissez la fréquence des captures en secondes entre 1 et 20 secondes : " ))
NombreDeCaptures = int(input("saisissez le nombre de captures souhaitées : "))
while NombreDeCaptures >4 or NombreDeCaptures <1 :
    NombreDeCaptures = int(input("saisissez le nombre de captures entre 1 et 4 : " ))
camera = PiCamera()
camera.resolution = (400, 300)
camera.start_preview()
# Camera warm-up time
sleep(2)
u=0
for i in range(NombreDeCaptures):
    u = str(i)+'.jpg'
#    print(u)
    camera.capture(u)
    shutil.copy(u,'/var/www/html/sites/media/images/', follow_symlinks=True)
    sleep(FrequenceCapture)
camera.stop_preview()
camera.close()

saisissez la fréquence des captures en secondes entre 1 et 20 secondes : 5
saisissez le nombre de captures souhaitées : 4


## essai en markdown 

|images|images|
|:---:|:---:|
|![image labo](http://172.18.161.5/sites/media/images/0.jpg)|![image labo](http://172.18.161.5/sites/media/images/1.jpg) |
|![image labo](http://172.18.161.5/sites/media/images/2.jpg)|![image labo](http://172.18.161.5/sites/media/images/3.jpg) |

In [ ]:
%%html
<p>insertion image</p>
<center>
<table align="center" border="2" cellpadding="2" cellspacing="2" style="height:100px;width:500px;">
	<caption>Captures</caption>
	<tbody>
		<tr>
			<td><img alt="" src="http://172.18.161.5/sites/media/images/0.jpg" style="border-width: 2px; border-style: solid; margin: 2px; width: 400px; height: 200px;" /></td>
			<td><img alt="" src="http://172.18.161.5/sites/media/images/1.jpg" style="border-width: 2px; border-style: solid; margin: 2px; width: 400px; height: 200px;" /></td>
		</tr>
		<tr>
			<td><img alt="" src="http://172.18.161.5/sites/media/images/1.jpg" style="border-width: 2px; border-style: solid; margin: 2px; width: 400px; height: 200px;" /></td>
			<td><img alt="" src="http://172.18.161.5/sites/media/images/3.jpg" style="border-width: 2px; border-style: solid; margin: 2px; width: 400px; height: 200px;" /></td>
		</tr>
	</tbody>
</table>
</center>
<center>&nbsp;</center>

<center>&nbsp;</center>

<p>&nbsp;</p>

# Envoyer un mail 

Pour alerter sur la présence, envoie de la dernière image.

In [3]:
import smtplib
 
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login("compte@gmail.com", "motdepasseenclair")
 
msg = "test depuis raspi"
server.sendmail("eric.lejan@gmail.com", "eric.lejan@mac.com", msg)
server.quit()

(221, b'2.0.0 closing connection e202sm11207016wma.43 - gsmtp')

In [16]:
import os
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart


def SendMail():
    img_data = open("/var/www/html/sites/media/images/3.jpg", 'rb').read()
    msg = MIMEMultipart()
    msg['Subject'] = 'test from raspi'
    msg['From'] = 'eric.lejan@gmail.com'
    msg['To'] = 'eric.lejan@mac.com'

    text = MIMEText("essai de piece jointe")
    msg.attach(text)
    image = MIMEImage(img_data, name=os.path.basename("/var/www/html/sites/media/images/3.jpg"))
    msg.attach(image)

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login("compte@gmail.com", "motdepasseenclair")
    s.sendmail("from", "to", msg.as_string())
    s.quit()

In [17]:
SendMail()

## On déclenche la capture quand le détecteur perçoit une présence. 

Rem : Ce programme plante car la version de l'image contient une erreur, elle est corrigée dans la dernière version.

Pour suivre une des contrainte du projet, les fonctions sont définies. 

Le champ ci-dessous importe dans le kernel les fonctions detection() capture() et le programme principal main() qui les exploite.

In [ ]:
#On imorte la bibliothèque de base de temps, la bibliothèque mraa (compatibilité avec RASPI, Arduino, Intel)
#La bibliothèque RPi.GPIO
#la bibliothèque shutil pour gérer les fichiers
#la bibliothèque os pour le principe
#la bibliothèque picamera

import time
import mraa
import shutil
import os
from picamera import PiCamera
import RPi.GPIO as io

# On décide d'utiliser les Numéro des pin inscrits sur le GPIO   
io.setmode(io.BOARD)

# on branche le détecteur PIR sur le pin 18

pir_pin = 18
# On defini le pin 11 pour la led (Rouge)
led = mraa.Gpio(11)
time.sleep(0.5)
#On définit le pin 32 pour la ledverte
ledVerte = mraa.Gpio(32)
#on place le pin 18 en mode IN    
io.setup(pir_pin, io.IN)         
#on place le pin 11 et le pin 32 en mode OUT
status = led.dir(mraa.DIR_OUT)
time.sleep(0.5)
statusVert = ledVerte.dir(mraa.DIR_OUT)

def capture():
    from picamera import PiCamera
    from time import sleep
    camera = PiCamera()
    camera.resolution = (1024, 768)
    camera.start_preview()
    # Camera warm-up time
    sleep(2)
    camera.capture('foo2.jpg')
    sleep(0.5)
    shutil.copy('foo2.jpg','/var/www/html/sites/media/images/foo2bis.jpg', follow_symlinks=True)

def Capture():
    
def detection():
    while True:
        if io.input(pir_pin):
            status = led.write(1)
            statusVert = ledVerte.write(0)
            print("Il y a quelqu'un !…… ", io.input(pir_pin) ,"…",end='')
            capture()
        else:
            status = led.write(0)
            statusVert = ledVerte.write(1)
            print("Il n'y a personne !…… ", io.input(pir_pin) ,"…",end='')
        time.sleep(1)


def main():
    while 1:
        try:
            detection()
        except KeyboardInterrupt:
            print ('Arrêté à :', time.strftime("%A %d %B %Y %H:%M:%S"))
            status = led.write(0)
            statusVert = ledVerte.write(0)
            break

In [ ]:
main()